# Webpage Creator

Task:
* Brainstorm ideas for webpages to build (e.g Pizza store, Gaming, Cooking etc)​.
* Prepare the `task` prompt​ with your idea.
* Run the code, and see your webpage!


In [ ]:
task: str = """\
A website that sells energy drinks called 'Boost'.
There are many different fruity flavours but the main one I want to sell is the red berry.

It should be saturated with red and have a modern, sleek design.
The website should include sections for product information, customer reviews,
and a blog about the benefits of energy drinks. It should also have a contact form and links to social media.

Make the UI responsive, and a single-page application (SPA).
"""
# task: str = "A website about video games, complete with a store, blog etc."
# task: str = "A website about cars."
# task: str = "Make a website about coffee."
# task: str = "Make a website about animals, such as the zoo or a safari."
# task: str = "Make a website related to pizza."
# task: str = "Make a website related to cooking."
# task: str = "Make a blog post about one of your passions."
# task: str = "Make a useful website for high school students looking to apply to university."


In [ ]:
ENABLE_BACKGROUND_IMAGE: bool = True

ENABLE_SITE_IMAGES: bool = True
NUM_SITE_IMAGES: int = 3


In [ ]:
coder_system_message: str = """Website Developer.
You are an expert in programming HTML. You are tasked with creating a website for a client.

Your response must only be a single "```html" block, and include all contents of the file.
Do not abbreviate the HTML tags, and ensure that the file is complete and valid.
"""

In [ ]:
critic_system_message: str = """Critic.
You are a helpful assistant highly skilled in evaluating the quality of a given webpage's HTML code by providing a score from 1 (bad) - 10 (good) while providing clear rationale.
YOU MUST CONSIDER HTML BEST PRACTICES for each evaluation. Specifically, you can carefully evaluate the code across the following dimensions
- Bugs (bugs): Are there logic errors, syntax error or typos? If ANY bug exists, the bug score MUST be less than 5.
- Goal compliance (compliance): How well the code meets the specified webpage goals?
- Navigation (navigation): Is the navigation bar easy to use and understand? Does it have a clear hierarchy? No navigation bar means a score of 0. No href links means a score of 0.
- Layout (layout): Is the layout of the webpage appropriate? Is the webpage responsive? Does it look good on mobile devices?
- Aesthetics (aesthetics): Is the webpage visually appealing? Does it have a good color scheme? Does it have a good font? Does it have a good use of whitespace? If there is no CSS, then mark this as 0.
- Images (images): Do images have a max-width so that they do not take up too much space? Make sure that `max-width: 50%` is the largest an image can be.
- Content (content): Suggest improvements to the content. Are more sections needed? Is it worded well?

YOU MUST PROVIDE A SCORE for each of the above dimensions.
{bugs: 0, compliance: 0, navigation: 0, layout: 0, aesthetics: 0, images: 0, content: 0}
Do not suggest code.
Finally, based on the critique above, suggest a concrete list of actions that the coder should take to improve the code.

After your scores and rationale, provide enhancements to the content that can be made.
"""

## Imports

In [ ]:
pip install -U openai pyautogen autogen


In [ ]:
# Maximum number of messages in the chat
MAX_MESSAGES: int = 5


In [ ]:
import os
import time
import base64

from openai import OpenAI, BadRequestError
from random import randint
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


In [ ]:
# List of OpenAI API configurations (Azure OpenAI)
import json

with open("OAI_CONFIG_LIST", "r") as f:
    config_list: list = json.load(f)

print(f"Using {len(config_list)} OpenAI API configurations")


In [ ]:
# Create model clients for the agents
coder_model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=config_list[0]["base_url"],
    api_key=config_list[0]["api_key"],
    api_version=config_list[0]["api_version"],
    model=config_list[0]["model"],
    azure_deployment=config_list[0]["model"],
    temperature=0.1,
)

critic_model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=config_list[0]["base_url"],
    api_key=config_list[0]["api_key"],
    api_version=config_list[0]["api_version"],
    model=config_list[0]["model"],
    azure_deployment=config_list[0]["model"],
    temperature=0.3,
)

print("Model clients created successfully")


## Setup Images

In [ ]:
from openai import AzureOpenAI
import os


# Azure OpenAI configuration


AZURE_OPENAI_ENDPOINT = config_list[0]["base_url"]
AZURE_OPENAI_API_KEY = config_list[0]["api_key"]
AZURE_DEPLOYMENT_NAME = config_list[0]["model"]
AZURE_OPENAI_VERSION = config_list[0]["api_version"]

print(f"Azure OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Azure Deployment Name: {AZURE_DEPLOYMENT_NAME}")
print(f"Azure OpenAI Version: {AZURE_OPENAI_VERSION}")

#model = "dall-e-3"

# Initialize Azure OpenAI client
client = AzureOpenAI(azure_endpoint=AZURE_OPENAI_ENDPOINT, azure_deployment=AZURE_DEPLOYMENT_NAME, api_version=AZURE_OPENAI_VERSION, api_key=AZURE_OPENAI_API_KEY)


#client = OpenAI(api_key=apikey)
#model = "dall-e-3"


In [ ]:
# Update the `get_image_filename` function to use Azure OpenAI
def get_image_filename(image_generation_prompt: str) -> str:
    """Generate a filename based on the image generation prompt, using Azure OpenAI"""
    prompt: str = f"Generate a filename for an image based on the following prompt:\n```{image_generation_prompt}```\n\nFilename: "

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates filenames for images that were generated using the given prompt."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=20,
        temperature=0.0
    )

    filename: str = response.choices[0].message.content.strip()
    filename2 = '.'.join(filename.split('.')[:-1])  # Remove any file extension if present
    filename3 = ''.join(c for c in filename2 if c.isalnum() or c in ('_', '-')).strip()
    assert filename3, "Filename cannot be empty.\n" + \
                      "Raw response message: `" + response.choices[0].message.content + "`\n" + \
                      "Filename 1: `" + filename + "`\n" + \
                      "Filename 2: `" + filename2 + "`\n" + \
                      "Filename 3: `" + filename3 + "`"
    filename = filename3

    number_suffix = 1
    while f"{filename}_{number_suffix}.png" in os.listdir("generated_images"):
        number_suffix += 1

    return f"{filename}_{number_suffix}.png"

# Update the `generate_image` function to use Azure OpenAI
def generate_image(image_generation_prompt: str, filename: str = None) -> str:
    """Generate an image based on the prompt using Azure OpenAI"""
    try:
        ## it should not be necessary
        client = AzureOpenAI(azure_endpoint=AZURE_OPENAI_ENDPOINT, azure_deployment="dall-e-3", api_version=AZURE_OPENAI_VERSION, api_key=AZURE_OPENAI_API_KEY)

        response = client.images.generate(
            model="dall-e-3",
            prompt=image_generation_prompt,
            size="1024x1024",
            response_format="b64_json"
        )
    except Exception as e:
        print(f"Error generating image: {e}")
        return

    img_data = response.data[0].b64_json
    img_bytes = base64.b64decode(img_data)

    if not filename:
        filename = get_image_filename(image_generation_prompt)

    print(f"Saving image as: `{filename}`...")
    with open(os.path.join('generated_images', filename), 'wb') as handler:
        handler.write(img_bytes)

# Update the `get_website_background_image_prompt` function to use Azure OpenAI
def get_website_background_image_prompt(task: str) -> str:
    """Generate a prompt for the background image of the website using Azure OpenAI"""
    prompt: str = f"What can be shown on the background of a website about:\n```{task}```\n\nPrompt: "

    system_prompt: str = """\
You are a helpful assistant that generates prompts for background images
to be displayed on websites. 
Ensure that the image is suitable for a website background (i.e. light, scenic, faded, indiscreet, minimalistic, abstract).
The image should not be distracting or detailed, and should complement the website's theme.
Prompts should be descriptive, not contain verbs and about 10-20 words long.
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=1.5
    )

    return response.choices[0].message.content.strip()

print("done")

In [ ]:
# Background image for the webpage
if ENABLE_BACKGROUND_IMAGE:
    # Background image for the webpage
    if ENABLE_BACKGROUND_IMAGE:
        background_image_prompt: str = get_website_background_image_prompt(task)
        print(f"Background image prompt: {background_image_prompt}")
        try:
            generate_image(image_generation_prompt=background_image_prompt, filename='background.png')
        except AttributeError as e:
            print(f"Error: {e}. Ensure the AzureOpenAI client is correctly configured.")


In [ ]:
# Generate new images that can be used on the webpage
def get_image_generation_prompt(task: str) -> str:
    """Create a new image generation prompt for the website"""
    prompt: str = f"What image can be shown on a website about:\n```{task}```\n\nPrompt: "

    system_prompt: str = """\
You are a helpful assistant that generates prompts for images to be displayed on websites.
Decide on one specific object to describe in the prompt. Be descriptive, exclude verbs and stick to 10-20 words long.
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=1.6,
        seed=randint(0, 10000000)  # Random seed to force different prompts each time
    )

    return response.choices[0].message.content.strip()


# Generate multiple images for the website
if ENABLE_SITE_IMAGES:
    for i in range(NUM_SITE_IMAGES):
        new_image_prompt: str = get_image_generation_prompt(task=task)
        print(f"Image generation prompt: `{new_image_prompt}`")
        generate_image(image_generation_prompt=new_image_prompt)
        time.sleep(1)  # Sleep for a second to avoid hitting the rate limit


In [ ]:
# Find images that can be used
def list_available_images() -> list:
    """List all available images."""
    default_images = [os.path.join("..", "images", image)
                      for image in os.listdir("images")
                      if image.endswith((".jpg", ".jpeg", ".png"))]
    generate_images = [os.path.join("..", "generated_images", image)
                       for image in os.listdir("generated_images")
                       if image.endswith((".jpg", ".jpeg", ".png"))]
    return default_images + generate_images

image_filenames = list_available_images()


print(f"Image file names: {image_filenames}")


## Shared Configs


In [ ]:
# Termination condition for the team
termination_condition = MaxMessageTermination(max_messages=MAX_MESSAGES) | TextMentionTermination("TERMINATE")


## Construct Agents

In [ ]:
# We'll store agents in a list to pass to the team
participants = []


In [ ]:
coder = AssistantAgent(
    name="Coder",
    model_client=coder_model_client,
    system_message=coder_system_message,
)

participants.append(coder)
print(f"Created agent: {coder.name}")


In [ ]:
critic = AssistantAgent(
    name="Critic",
    model_client=critic_model_client,
    system_message=critic_system_message,
)

participants.append(critic)
print(f"Created agent: {critic.name}")


### User Proxy Agent


In [ ]:
# Note: UserProxyAgent in the new API doesn't require the same configuration
# The team will handle message routing automatically
print(f"Total participants: {len(participants)}")


### Create Group Chat

In [ ]:
print("Chat participants:")
for agent in participants:
    print(f" - {agent.name}")


In [ ]:
# Create the RoundRobinGroupChat team
team = RoundRobinGroupChat(
    participants=participants,
    termination_condition=termination_condition,
)

print(f"Created RoundRobinGroupChat team with {len(participants)} participants")


## Setup Conversation

In [ ]:
system_prompt: str = f"""\
Build a responsive HTML website for the client.

Client's request:

```
{task.strip()}
```
"""

images_list_str = '\n'.join(image_filenames)
if images_list_str:
    system_prompt += f"""
Here are photos that are available for use:
{images_list_str}

Be selective with the images you use. You do not need to use them at all.
"""
    
if ENABLE_BACKGROUND_IMAGE:
    system_prompt += "\nUse the background.png image as the background for the website, with a transparency of 50%."

print(images_list_str)

## Start Conversation

In [ ]:
# Run the team with the system prompt
import asyncio

async def run_team():
    result = await team.run(task=system_prompt)
    return result

# Run the async function
result = await run_team()
print("\nConversation completed!")


## Save Webpage


In [ ]:
webpage_content: str = """\
<!DOCTYPE html>
<html>
<head>
    <title>Not Found</title>
</head>
<body>
    <p>HTML could not be extracted from the chat.</p>
</body>
"""

def extract_html_from_message(message_content: str) -> str:
    start_delimiter = "```html"
    end_delimiter = "```"
    start_index = message_content.find(start_delimiter)
    if start_index == -1:
        return None
    end_index = message_content.find(end_delimiter, start_index + len(start_delimiter))
    if end_index == -1:
        return None
    return message_content[start_index + len(start_delimiter):end_index]

# Extract HTML from the result messages
if hasattr(result, 'messages'):
    for message in result.messages:
        if hasattr(message, 'content') and isinstance(message.content, str):
            if "```html" in message.content:
                extracted = extract_html_from_message(message.content)
                if extracted:
                    webpage_content = extracted
                    print(f"Found HTML in message from: {message.source if hasattr(message, 'source') else 'unknown'}")
                    break

print(f"Webpage content length: {len(webpage_content)} characters")


In [ ]:
def get_webpage_filename(task: str) -> str:
    """Generate a filename based on the webpage prompt, using OpenAI"""
    prompt: str = f"Provide a filename for a HTML file that was generated using the prompt:\n```{task}```\n\nFilename: "

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates filenames "
                                          "for HTML files that were generated using the given prompt."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=20,
        temperature=0.0
    )

    filename: str = response.choices[0].message.content.strip()
    filename = '.'.join(filename.split('.')[:-1])  # Remove any file extension if present
    # Only allow alphanumeric characters, underscores, and dashes in the filename
    filename = ''.join(c for c in filename if c.isalnum() or c in ('_', '-')).strip()

    number_suffix = 1
    while f"{filename}_{number_suffix}.html" in os.listdir("output"):
        number_suffix += 1

    return f"{filename}_{number_suffix}.html"


In [ ]:
webpage_filename: str = get_webpage_filename(task)
print(f"Webpage filename: `{webpage_filename}`")
webpage_path: str = os.path.join("output", webpage_filename)

with open(webpage_path, "w") as f:
    f.write(webpage_content)
    print(f"Webpage created at `{webpage_path}`")


## Open Webpage in browser


In [ ]:
# Windows: Open webpage in browser
# import webbrowser
# webbrowser.open(webpage_path, new=1)

# MacOS: Open webpage in browser
import os
os.system(f"open {webpage_path}")

# MacOS: Alternate method to open webpage in browser
# import subprocess
# subprocess.run(["open", webpage_path])


## Clean up files

In [ ]:
import os

def delete_files_in_folder(folder_path: str):
    """Delete all files in the specified folder."""
    if os.path.exists(folder_path):
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Deleted: {file_path}")
    else:
        print(f"Folder does not exist: {folder_path}")

# Paths to the folders
generated_images_folder = "generated_images"
output_folder = "output"

# Delete files in the folders
delete_files_in_folder(generated_images_folder)
delete_files_in_folder(output_folder)

In [ ]:
print("DONE")
